# Loop 43 LB Feedback Analysis

**exp_043 (subset_extraction)**: CV 70.3070 | LB 70.3070 (gap: +0.0000)

## Key Findings
1. **CV-LB gap is ZERO** - Our local validation is perfectly calibrated
2. **Subset extraction found 2 improvements** - N=121 and N=122
3. **Total improvement**: 0.001612 points
4. **Gap to target**: 1.446 points (2.05%)

In [1]:
# Current state analysis
import json

with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

# Count experiment types
experiments = state['experiments']
print(f"Total experiments: {len(experiments)}")
print(f"Submissions used: {21}/100")
print(f"Remaining: 79 submissions")

# Best scores
best_cv = min(exp.get('cv_score', exp.get('score', 999)) for exp in experiments)
print(f"\nBest CV: {best_cv:.6f}")
print(f"Target: 68.861114")
print(f"Gap: {best_cv - 68.861114:.6f} ({(best_cv - 68.861114) / 68.861114 * 100:.2f}%)")

# Recent progress
print("\nRecent experiments:")
for exp in experiments[-5:]:
    cv = exp.get('cv_score', exp.get('score', 'N/A'))
    fallback = exp.get('used_baseline_fallback', False)
    print(f"  {exp['name'][:40]:40s} CV: {cv:.6f} {'FALLBACK' if fallback else ''}")

Total experiments: 44
Submissions used: 21/100
Remaining: 79 submissions

Best CV: 70.265730
Target: 68.861114
Gap: 1.404616 (2.04%)

Recent experiments:
  039_per_n_gap_analysis                   CV: 70.308629 
  040_extended_mining                      CV: 70.308619 
  041_constructive_small_n                 CV: 70.308629 FALLBACK
  042_row_based_constructive               CV: 70.308629 FALLBACK
  043_subset_extraction                    CV: 70.307017 


## Critical Analysis

### What's Working
1. **Subset extraction** - First algorithmic improvement in 10+ experiments
2. **CV-LB calibration** - Perfect match, no validation issues
3. **External data mining** - Found improvements from multiple sources

### What's NOT Working
1. **Local search methods** (SA, bbox3, shake) - ZERO improvement
2. **Constructive approaches** - WORSE than baseline (fallback)
3. **Genetic algorithm** - ZERO improvement

### The Gap Problem
- Current: 70.307
- Target: 68.861
- Gap: 1.446 points (2.05%)

**This gap is TOO LARGE for incremental improvements!**

Subset extraction found 0.001612 improvement. At this rate:
- Need ~900 similar improvements to reach target
- But subset extraction only found 2 improvements total

### What Top Teams Did Differently
1. **953 submissions** - Accumulated improvements over time
2. **Novel algorithms** - Not just running bbox3/SA
3. **Per-N specialization** - Different approaches for different N ranges

In [2]:
# Analyze what N values have the most room for improvement
import pandas as pd
import numpy as np

df = pd.read_csv('/home/submission/submission.csv')

def parse_coord(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
df['x'] = df['x'].apply(parse_coord)
df['y'] = df['y'].apply(parse_coord)
df['deg'] = df['deg'].apply(parse_coord)

# Tree polygon vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_vertices(x, y, angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rx = TX * cos_a - TY * sin_a
    ry = TX * sin_a + TY * cos_a
    return rx + x, ry + y

def compute_bbox_size(trees):
    all_x, all_y = [], []
    for x, y, angle in trees:
        vx, vy = get_tree_vertices(x, y, angle)
        all_x.extend(vx)
        all_y.extend(vy)
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def compute_score(trees, n):
    size = compute_bbox_size(trees)
    return (size ** 2) / n

# Compute per-N scores
per_n_scores = {}
for n in range(1, 201):
    n_df = df[df['n'] == n]
    trees = [(row['x'], row['y'], row['deg']) for _, row in n_df.iterrows()]
    per_n_scores[n] = compute_score(trees, n)

print("Per-N score analysis:")
print(f"Total: {sum(per_n_scores.values()):.6f}")
print(f"\nTop 10 contributors:")
for n, score in sorted(per_n_scores.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  N={n:3d}: {score:.6f} ({score/sum(per_n_scores.values())*100:.2f}%)")

Per-N score analysis:
Total: 70.307017

Top 10 contributors:
  N=  1: 0.661250 (0.94%)
  N=  2: 0.450779 (0.64%)
  N=  3: 0.434745 (0.62%)
  N=  5: 0.416850 (0.59%)
  N=  4: 0.416545 (0.59%)
  N=  7: 0.399842 (0.57%)
  N=  6: 0.399610 (0.57%)
  N=  8: 0.385407 (0.55%)
  N=  9: 0.383042 (0.54%)
  N= 10: 0.376630 (0.54%)


## Strategy for Next Experiment

### EXPAND SUBSET EXTRACTION (Evaluator's Recommendation)

The subset extraction approach is the **ONLY** method that found improvements in 10+ experiments.

**What to try:**
1. **N+2 → N extraction**: Remove 2 trees from N+2 to create N
2. **N+3 → N extraction**: Remove 3 trees from N+3 to create N
3. **Cross-N extraction**: Use trees from multiple different N values
4. **Recursive extraction**: Apply extraction to the improved solutions

### Alternative: External Data Mining

Check for new kernels/datasets that might have better solutions:
- New kernels published in last 24 hours?
- New datasets shared?
- Solutions from other teams?

### The Math

To reach target (68.861), we need 1.446 points improvement:
- If we find 0.01 improvement per N value: need 145 N values
- If we find 0.001 improvement per N value: need 1446 N values (impossible)

**Conclusion**: Need LARGER improvements per N, not more tiny improvements.

In [ ]:
# Record key finding
print("KEY FINDING:")
print("="*60)
print("Subset extraction is the ONLY approach that found improvements")
print("in the last 10+ experiments.")
print("")
print("exp_043 found 2 improvements totaling 0.001612 points:")
print("  - N=121: +0.000766 (from N=122)")
print("  - N=122: +0.000846 (from N=123)")
print("")
print("This proves the baseline is NOT globally optimal for all N.")
print("The key is to find MORE improvements through expanded extraction.")
print("="*60)